In [4]:
import xarray as xr
import pandas as pd

In [5]:

dfA= xr.open_dataset('/net/archive/data/ace-1/aircraft/c130_n130ar/LRT/V1.1/RF31A.19951218.185659_225432.PNI.nc')
dfB= xr.open_dataset('/net/archive/data/ace-1/aircraft/c130_n130ar/LRT/V1.1/RF31B.19951218.231256_042711.PNI.nc')

In [6]:
starta='1995-12-18 18:56:59'
startb='1995-12-18 23:12:56'
interval='18:56:59-04:27:11'


In [7]:
Fdate=df.attrs['FlightDate'].split('/') 
Fdate=Fdate[2]+'-'+Fdate[0]+'-'+Fdate[1]
Fdate+' '+df.attrs['TimeInterval'].split('-')[0]

'1995-10-31 16:05:22'

In [8]:
#convert time from seconds since midnight to datetime
def convert_time(df, start):
    return pd.to_datetime(df.Time, unit='s', origin=start)



In [9]:
def merge_datasets(dfA, dfB,starta, startb, flight, interval):
    startB = pd.Timestamp(startb)
    startA=pd.Timestamp(starta)##'1995-11-02 18:55:21'
    reference_time = startA.normalize() #normalize to midnight
    dfB['Time']=convert_time(dfB, startB)
    dfA['Time']=convert_time(dfA, startA)
    #dfB['time_offset'] = dfB['time_offset'] +dfA['base_time']
    merged = xr.concat([dfA, dfB], dim='Time')
    complete_time_index = pd.date_range(start=min(merged['Time'].min().values, merged['Time'].min().values),
                                        end=max(merged['Time'].max().values, merged['Time'].max().values),
                                        freq='s')
    merged = merged.reindex({'Time': complete_time_index})
    encoding = {var: {'_FillValue': int(-32767)} for var in merged.variables}
    merged['Time']=(merged['Time'] - reference_time.to_datetime64()) / pd.Timedelta(seconds=1)
    merged =merged.drop('base_time')
    merged['base_time']=dfA.base_time
    merged['time_offset'].values = range(len(merged.time_offset))
    merged.Time.attrs['long_name']='time of measurement'
    merged.Time.attrs['units']='seconds since'+reference_time.strftime('%Y-%m-%d %H:%M:%S +0000%Z')
    merged.Time.attrs['standard_name']='time'
    merged.Time.attrs['strptime_format']="seconds since %F %T %z" 
    merged.attrs['TimeInterval'] = interval
    merged.attrs['history']=merged.attrs['history'] + '\nWed Dec 12 Merged RF31A and RF31B'
    merged = merged.fillna(-32767)
    merged.to_netcdf(f'/scr/raf_Raw_Data/ACE-1/mergeLRT/ACE-1{flight}.nc', encoding=encoding)

In [10]:
merge_datasets(dfA, dfB,starta, startb, 'rf31', interval)

KeyboardInterrupt: 

In [ ]:
reference_time.strftime('%Y-%m-%d %H:%M:%S +0000%Z')

'1995-11-02 00:00:00 +0000'

In [11]:
##loop through files in directory
import os
directory = '/net/archive/data/ace-1/aircraft/c130_n130ar/LRT/V1.1/'
for file in os.listdir(directory):
    if file.endswith('.nc'):
        if 'RF02' in file or 'RF31' in file:
            continue
        
        df = xr.open_dataset(directory+file)
        Fdate=df.attrs['FlightDate'].split('/') 
        Fdate=Fdate[2]+'-'+Fdate[0]+'-'+Fdate[1]
        startT=pd.Timestamp(Fdate+' '+df.attrs['TimeInterval'].split('-')[0])
        reference_time = startT.normalize() #normalize to midnight
        df['Time']=convert_time(df, startT)
        
        df['Time']=(df['Time'] - reference_time.to_datetime64()) / pd.Timedelta(seconds=1)
        #merged =merged.drop('base_time')
        df.Time.attrs['long_name']='time of measurement'
        df.Time.attrs['units']='seconds since '+reference_time.strftime('%Y-%m-%d %H:%M:%S +0000%Z')
        df.Time.attrs['standard_name']='time'
        df.Time.attrs['strptime_format']="seconds since %F %T %z" 
        #df.attrs['TimeInterval'] = interval
        encoding = {var: {'_FillValue': int(-32767)} for var in df.data_vars}
        df.attrs['history']=df.attrs['history'] + '\nWed Dec 12 added Time variable'
        df = df.fillna(-32767)
        df.to_netcdf(f'/scr/raf_Raw_Data/ACE-1/mergeLRT/{file}', encoding=encoding)
            

In [ ]:
df.Time

<xarray.DataArray 'Time' (Time: 33817)>
array([57922., 57923., 57924., ..., 91736., 91737., 91738.])
Coordinates:
  * Time     (Time) float64 5.792e+04 5.792e+04 ... 9.174e+04 9.174e+04
Attributes:
    long_name:        time of measurement
    units:            seconds since1995-10-31 00:00:00 +0000
    standard_name:    time
    strptime_format:  seconds since %F %T %z

In [ ]:
merged['time_offset'].values = range(len(merged.time_offset))

In [ ]:
merged.attrs['history']=merged.attrs['history'] + '\nWed Dec 12 Merged RF02A and RF02B'

In [ ]:
encoding = {var: {'_FillValue': int(-32767)} for var in merged.variables}
merged.to_netcdf('/scr/raf_Raw_Data/ACE-1/mergeLRT/ACE-1rf02.nc', encoding=encoding)

In [ ]:
merged.to_netcdf('/scr/raf_Raw_Data/ACE-1/mergeLRT/ACE-1rf02.nc', encoding=encoding)

/h/eol/srunkel/.conda/envs/animation/lib/python3.12/site-packages/xarray/coding/times.py:618: RuntimeWarning: invalid value encountered in cast
  int_num = np.asarray(num, dtype=np.int64)


In [ ]:
merged.attrs

{'Source': 'NCAR Research Aviation Facility',
 'Address': 'P.O. Box 3000, Boulder, CO 80307-3000',
 'Phone': '(303) 497-1030',
 'Conventions': 'NCAR-RAF/nimbus',
 'DateProcessed': 'Apr 25 14:51 UTC 1996',
 'ProjectName': 'ACE - 1',
 'Aircraft': 'N130AR',
 'ProjectNumber': '130',
 'FlightNumber': '2',
 'FlightDate': '11/02/1995',
 'TimeInterval': '19951102T18:55:21-19951103T02:14:30',
 'Defaults': 'CNDIV=1.027000e+03,3.000000e+01,1.562000e+03 : O3FSM_CAL=4.070000e+01 : XNOYM_CAL=-2.000000e+01,5.200000e+00 : XNOM_CAL=-1.000000e+01,4.300000e+01 : XNO2C_CAL=9.580000e+02 : C2T=3.094500e+02 : CO2_CAL=1.520700e-01 : H2T=3.146500e+02 : H2O_CAL=7.135200e-03 : TWIRE_PMS=1.550000e+02 : TWIRE_PMS1=1.450000e+02 : PITCH_BIAS_1=0.000000e+00 : HEADING_BIAS_1=-4.000000e-01 : PITCH_BIAS_2=0.000000e+00 : HEADING_BIAS_2=0.000000e+00 : C1_EAW=8.000000e-01 : C2_EAW=8.000000e-01 : P1_EAW=6.400000e+00 : P2_EAW=3.200000e+00 : XC_1=5.100000e-01 : THRSH_1=9.000000e+02 : CX_1=9.770000e+00,1.297400e+00,4.102000e-0

In [ ]:
#I want to merge StartA and StartB but fill all the missing time values in between with nans


In [ ]:
merged.base_time

In [ ]:
# Create a complete time index
xr.open_dataset('/scr/raf_data/CAESAR/CAESARrf01.nc')

<xarray.Dataset>
Dimensions:                 (Time: 25065, F2DC003: 64, bnds: 2, sps1: 1,
                             F2DS020: 13, histogram98: 98, CDP016: 30,
                             CDP058: 30, HVPS315: 13, PCAS108: 30,
                             UHSAS011: 99, interarrival_endpoints: 40,
                             Vector128: 128, Vector256: 256)
Coordinates:
  * Time                    (Time) datetime64[ns] 2024-02-28T11:39:00 ... 202...
  * F2DC003                 (F2DC003) float32 25.0 50.0 ... 1.575e+03 1.6e+03
  * F2DS020                 (F2DS020) float32 9.969e+36 9.969e+36 ... 9.969e+36
  * CDP016                  (CDP016) float32 2.42 3.4 4.38 ... 44.08 46.04 48.0
  * CDP058                  (CDP058) float32 1.835 2.865 3.895 ... 47.7 49.76
  * HVPS315                 (HVPS315) float32 9.969e+36 9.969e+36 ... 9.969e+36
  * PCAS108                 (PCAS108) float32 0.101 0.109 0.117 ... 2.537 2.826
  * UHSAS011                (UHSAS011) float32 0.06086 0.06262 ... 0.9584 0.986
  * interarrival_endpoints  (interarrival_endpoints) float32 9.969e+36 ... 9....
Dimensions without coordinates: bnds, sps1, histogram98, Vector128, Vector256
Data variables: (12/715)
    F2DC003_bnds            (F2DC003, bnds) float32 ...
    A1DC_RPC                (Time, sps1, F2DC003) float32 ...
    F2DS020_bnds            (F2DS020, bnds) float32 ...
    A2DSH_2DS               (Time, sps1, F2DS020) float32 ...
    A2DSV_2DS               (Time, sps1, F2DS020) float32 ...
    A2DTEMP_GPDACQ_319      (Time) float32 ...
    ...                      ...
    REFF2DHR_H1             (Time) float32 ...
    REFF2DHA_H1             (Time) float32 ...
    NACCEPT2DHR_H1          (Time) float32 ...
    NACCEPT2DHA_H1          (Time) float32 ...
    NREJECT2DHR_H1          (Time) float32 ...
    NREJECT2DHA_H1          (Time) float32 ...
Attributes: (12/52)
    program:                        NSF NCAR
    institution:                    National Center for Atmospheric Research
    Address:                        P.O. Box 3000, Boulder, CO 80307-3000
    Phone:                          (303) 497-1030
    source:                         airborne observations
    platform:                       N130AR
    ...                             ...
    geospatial_lon_max:             20.608355
    geospatial_vertical_min:        169.68057
    geospatial_vertical_max:        5996.425
    time_coverage_start:            2024-02-28T11:39:00 +0000
    time_coverage_end:              2024-02-28T18:36:44 +0000
    TimeInterval:                   11:39:00-18:36:44

'18:55:21-00:00:09'

In [ ]:
##Now that it's merged, I want to write it with fillvalue -32767 and time as seconds since midnight
merged.to_netcdf('/scr/raf_Raw_Data/ACE-1/mergeLRT/ACE-1rf01.nc', encoding=encoding)

/h/eol/srunkel/.conda/envs/animation/lib/python3.12/site-packages/xarray/coding/times.py:618: RuntimeWarning: invalid value encountered in cast
  int_num = np.asarray(num, dtype=np.int64)


In [ ]:
#mdinight of first day
#Update time offset for fileB
### /scr/temp/bansemer BULK file will not be used

In [ ]:
xr.open_dataset('/scr/raf_Raw_Data/ACE-1/mergeLRT/ACE-1rf01.nc')['Time']

<xarray.DataArray 'Time' (Time: 26350)>
array(['1995-11-02T18:55:21.000000000', '1995-11-02T18:55:22.000000000',
       '1995-11-02T18:55:23.000000000', ..., '1995-11-03T02:14:28.000000000',
       '1995-11-03T02:14:29.000000000', '1995-11-03T02:14:30.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * Time     (Time) datetime64[ns] 1995-11-02T18:55:21 ... 1995-11-03T02:14:30

In [ ]:
merged.sel(Time=slice('1995-11-03 00:00:09', '1995-11-03 00:04:30'))

<xarray.Dataset>
Dimensions:      (Time: 262, sps1: 1, Vector64: 64, Vector32: 32, Vector16: 16)
Coordinates:
  * Time         (Time) datetime64[ns] 1995-11-03T00:00:09 ... 1995-11-03T00:...
Dimensions without coordinates: sps1, Vector64, Vector32, Vector16
Data variables: (12/256)
    base_time    (Time) float64 8.153e+08 nan nan ... 8.154e+08 8.154e+08
    time_offset  (Time) float32 1.829e+04 nan nan nan ... 13.0 14.0 15.0 16.0
    ADIFR        (Time) float32 -15.15 nan nan nan ... -15.76 -15.79 -15.69
    AZFLOW       (Time) float32 -0.1171 nan nan nan ... -0.1237 -0.1221 -0.1198
    AZSAMP1      (Time) float32 -0.000722 nan nan ... -0.0008022 -0.0003209
    AZSAMP2      (Time) float32 0.005942 nan nan ... 0.004416 0.004898 0.004737
    ...           ...
    XVYC         (Time) float32 -10.59 nan nan nan ... -10.83 -10.86 -10.84
    XWDC         (Time) float32 139.6 nan nan nan ... 140.5 140.6 140.6 140.7
    XWIC         (Time) float32 -0.8857 nan nan nan ... -0.764 -0.8246 -0.8311
    XWSC         (Time) float32 18.53 nan nan nan ... 19.49 19.43 19.47 19.47
    ZERO         (Time) float32 0.0 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    O3FV         (Time) float32 155.9 nan nan nan ... 156.1 155.5 155.7 156.9
Attributes: (12/16)
    Source:                         NCAR Research Aviation Facility
    Address:                        P.O. Box 3000, Boulder, CO 80307-3000
    Phone:                          (303) 497-1030
    Conventions:                    NCAR-RAF/nimbus
    DateProcessed:                  Apr 25 14:51 UTC 1996
    ProjectName:                    ACE - 1
    ...                             ...
    TimeInterval:                   18:55:21-00:00:09
    Defaults:                       CNDIV=1.027000e+03,3.000000e+01,1.562000e...
    DOI:                            https://doi.org/10.5065/D6F18X1Z
    history:                        Mon Aug 12 15:51:00 2024: ncatted -O -a H...
    NCO:                            netCDF Operators version 5.0.7 (Homepage ...
    SizeDistributionLegacyZeroBin:  yes

In [ ]:
dfA

<xarray.Dataset>
Dimensions:      (Time: 18289, sps1: 1, Vector64: 64, Vector32: 32, Vector16: 16)
Dimensions without coordinates: Time, sps1, Vector64, Vector32, Vector16
Data variables: (12/256)
    base_time    float64 ...
    time_offset  (Time) float32 ...
    ADIFR        (Time) float32 ...
    AZFLOW       (Time) float32 ...
    AZSAMP1      (Time) float32 ...
    AZSAMP2      (Time) float32 ...
    ...           ...
    XVYC         (Time) float32 ...
    XWDC         (Time) float32 ...
    XWIC         (Time) float32 ...
    XWSC         (Time) float32 ...
    ZERO         (Time) float32 ...
    O3FV         (Time) float32 ...
Attributes: (12/16)
    Source:                         NCAR Research Aviation Facility
    Address:                        P.O. Box 3000, Boulder, CO 80307-3000
    Phone:                          (303) 497-1030
    Conventions:                    NCAR-RAF/nimbus
    DateProcessed:                  Apr 25 14:51 UTC 1996
    ProjectName:                    ACE - 1
    ...                             ...
    TimeInterval:                   18:55:21-00:00:09
    Defaults:                       CNDIV=1.027000e+03,3.000000e+01,1.562000e...
    DOI:                            https://doi.org/10.5065/D6F18X1Z
    history:                        Mon Aug 12 15:51:00 2024: ncatted -O -a H...
    NCO:                            netCDF Operators version 5.0.7 (Homepage ...
    SizeDistributionLegacyZeroBin:  yes